<a href="https://colab.research.google.com/github/MegaChie/AirBnB_clone/blob/main/Notebook/0x00_AirBnB_clone_The_console.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classes and Codes

In [ ]:
#!/usr/bin/python3
""" BaseModel """


import uuid
from datetime import datetime
# from models import storage


class BaseModel:
    """ BaseModel """
    def __init__(self, *args, **kwargs):
        if kwargs is not None and kwargs != {}:
            for key in kwargs:
                if key == "created_at":
                    self.__dict__["created_at"] = datetime.strptime(
                        kwargs["created_at"], "%Y-%m-%dT%H:%M:%S.%f")
                elif key == "updated_at":
                    self.__dict__["updated_at"] = datetime.strptime(
                        kwargs["updated_at"], "%Y-%m-%dT%H:%M:%S.%f")
                else:
                    self.__dict__[key] = kwargs[key]
        else:
            self.id = str(uuid.uuid4())
            self.created_at = datetime.now()
            self.updated_at = datetime.now()
            storage.new(self)

    def __str__(self):
        """ BaseModel """
        return "[{}] ({}) {}".\
            format(type(self).__name__, self.id, self.__dict__)

    def save(self):
        """ BaseModel """
        self.updated_at = datetime.now()
        storage.save()

    def to_dict(self):
        """ BaseModel """
        newDict = self.__dict__.copy()
        newDict["__class__"] = type(self).__name__
        newDict["created_at"] = newDict["created_at"].isoformat()
        newDict["updated_at"] = newDict["updated_at"].isoformat()
        return newDict

In [ ]:
#!/usr/bin/python3
""" FileStorage """


import json
import datetime
import os


class FileStorage:
    """ Store first object """
    __file_path = "file.json"
    __objects = {}

    def all(self):
        """ Store first object """
        return FileStorage.__objects

    def new(self, obj):
        """ Store first object """
        key = "{}.{}".format(type(obj).__name__, obj.id)
        FileStorage.__objects[key] = obj

    def save(self):
        """ Store first object """
        with open(FileStorage.__file_path, "w", encoding="utf-8") as writer:
            d = {key: value.to_dict() for
                 key, value in FileStorage.__objects.items()}
            json.dump(d, writer)

    def classes(self):
        # from models.base_model import BaseModel
        # from models.user import User
        # from models.state import State
        # from models.city import City
        # from models.amenity import Amenity
        # from models.place import Place
        # from models.review import Review
        classes = {"BaseModel": BaseModel,
                   "User": User,
                   "State": State,
                   "City": City,
                   "Amenity": Amenity,
                   "Place": Place,
                   "Review": Review}
        return classes

    def reload(self):
        """ Store first object """
        if not os.path.isfile(FileStorage.__file_path):
            return
        with open(FileStorage.__file_path, "r", encoding="utf-8") as opener:
            objDict = json.load(opener)
            objDict = {k: self.classes()[v["__class__"]](**v)
                       for k, v in objDict.items()}
            FileStorage.__objects = objDict

    def attributes(self):
        """ returns the valid attributes """
        attributes = {
            "BaseModel":
                     {"id": str,
                      "created_at": datetime.datetime,
                      "updated_at": datetime.datetime},
            "User":
                     {"email": str,
                      "password": str,
                      "first_name": str,
                      "last_name": str},
            "State":
                     {"name": str},
            "City":
                     {"state_id": str,
                      "name": str},
            "Amenity":
                     {"name": str},
            "Place":
                     {"city_id": str,
                      "user_id": str,
                      "name": str,
                      "description": str,
                      "number_rooms": int,
                      "number_bathrooms": int,
                      "max_guest": int,
                      "price_by_night": int,
                      "latitude": float,
                      "longitude": float,
                      "amenity_ids": list},
            "Review":
            {"place_id": str,
                         "user_id": str,
                         "text": str}
        }
        return attributes

In [ ]:
#!/usr/bin/python3
""" FileStorage """


# from models.engine.file_storage import FileStorage
storage = FileStorage()
storage.reload()

In [ ]:
#!/usr/bin/python3
""" First User """
# from models.base_model import BaseModel


class User(BaseModel):
    """ First User """
    email = ""
    password = ""
    first_name = ""
    last_name = ""


In [ ]:
#!/usr/bin/python3
""" More classes! """
# from models.base_model import BaseModel


class State(BaseModel):
    """ More classes! """
    name = ""


In [ ]:
#!/usr/bin/python3
""" More classes! """
# from models.base_model import BaseModel


class City(BaseModel):
    """ More classes! """
    state_id = ""
    name = ""


In [ ]:
#!/usr/bin/python3
""" More classes! """
# from models.base_model import BaseModel


class Amenity(BaseModel):
    """ More classes! """
    name = ""


In [ ]:
#!/usr/bin/python3
""" More classes! """
# from models.base_model import BaseModel


class Place(BaseModel):
    """ More classes! """
    city_id = ""
    user_id = ""
    name = ""
    name = ""
    number_rooms = 0
    number_bathrooms = 0
    max_guest = 0
    price_by_night = 0
    latitude = 0.0
    longitude = 0.0
    amenity_ids = []


In [ ]:
#!/usr/bin/python3
""" More classes! """
# from models.base_model import BaseModel


class Review(BaseModel):
    """ More classes! """
    place_id = ""
    user_id = ""
    text = ""


In [ ]:
#!/usr/bin/python3
""" Console """

import cmd
import json
import re
# from models.base_model import BaseModel
# from models import storage


class HBNBCommand(cmd.Cmd):
    prompt = "(hbnb) "

    def default(self, line):
        """ if all fails """
        self._precmd(line)

    def _precmd(self, line):
        """ intercepts commands """
        match = re.search(r"^(\w*)\.(\w+)(?:\(([^)]*)\))$", line)
        if not match:
            return line
        classname = match.group(1)
        method = match.group(2)
        args = match.group(3)
        match_uid_and_args = re.search('^"([^"]*)"(?:, (.*))?$', args)
        if match_uid_and_args:
            uid = match_uid_and_args.group(1)
            attr_or_dict = match_uid_and_args.group(2)
        else:
            uid = args
            attr_or_dict = False
        attr_and_value = ""
        if method == "update" and attr_or_dict:
            match_dict = re.search('^({.*})$', attr_or_dict)
            if match_dict:
                self.update_dict(classname, uid, match_dict.group(1))
                return ""
            match_attr_and_value = re.search(
                '^(?:"([^"]*)")?(?:, (.*))?$', attr_or_dict)
            if match_attr_and_value:
                attr_and_value = (match_attr_and_value.group(
                    1) or "") + " " + (match_attr_and_value.group(2) or "")
        command = method + " " + classname + " " + uid + " " + attr_and_value
        self.onecmd(command)
        return command

    def update_dict(self, classname, uid, s_dict):
        """ Helper method for update() """
        s = s_dict.replace("'", '"')
        d = json.loads(s)
        if not classname:
            print("** class name missing **")
        elif classname not in storage.classes():
            print("** class doesn't exist **")
        elif uid is None:
            print("** instance id missing **")
        else:
            key = "{}.{}".format(classname, uid)
            if key not in storage.all():
                print("** no instance found **")
            else:
                attributes = storage.attributes()[classname]
                for attribute, value in d.items():
                    if attribute in attributes:
                        value = attributes[attribute](value)
                    setattr(storage.all()[key], attribute, value)
                storage.all()[key].save()

    def do_quit(self, line):
        """ exit the program """
        return True

    def do_EOF(self, line):
        """ exit the program """
        return True

    def emptyline(self):
        """ does nothing """
        pass

    def do_create(self, line):
        """ create class instance """
        argument = line.split(' ')
        if argument is None or argument[0] == "":
            print("** class name missing **")
        elif argument[0] not in storage.classes():
            print("** class doesn't exist **")
        else:
            newClass = storage.classes()[argument[0]]()
            newClass.save()
            print(newClass.id)

    def do_show(self, line):
        """  Prints the string representation of an instance """
        argument = line.split(' ')
        if argument is None or argument[0] == "":
            print("** class name missing **")
        else:
            if argument[0] not in storage.classes():
                print("** class doesn't exist **")
            elif len(argument) < 2:
                print("** instance id missing **")
            else:
                key = "{}.{}".format(argument[0], argument[1])
                if key not in storage.all():
                    print("** no instance found **")
                else:
                    print(storage.all()[key])

    def do_destroy(self, line):
        """  Deletes an instance """
        argument = line.split(' ')
        if argument is None or argument[0] == "":
            print("** class name missing **")
        else:
            if argument[0] not in storage.classes():
                print("** class doesn't exist **")
            elif len(argument) < 2:
                print("** instance id missing **")
            else:
                key = "{}.{}".format(argument[0], argument[1])
                if key not in storage.all():
                    print("** no instance found **")
                else:
                    del storage.all()[key]
                    storage.save()

    def do_all(self, line):
        """ Prints all string representation of all instances """
        if line != "":
            argument = line.split(' ')
            if argument[0] not in storage.classes():
                print("** class doesn't exist **")
            else:
                text = [str(obj) for key, obj in storage.all().items()
                        if type(obj).__name__ == argument[0]]
                print(text)
        else:
            new_list = [str(obj) for key, obj in storage.all().items()]
            print(new_list)

    def do_count(self, line):
        """ Counts the instances of a class """
        argument = line.split(' ')
        if not argument[0]:
            print("** class name missing **")
        elif argument[0] not in storage.classes():
            print("** class doesn't exist **")
        else:
            matches = [
                k for k in storage.all() if k.startswith(
                    words[0] + '.')]
            print(len(matches))

    def do_update(self, line):
        """ Updates an instance by adding or updating attribute """
        if line == "" or line is None:
            print("** class name missing **")
            return
        rex = r'^(\S+)(?:\s(\S+)(?:\s(\S+)(?:\s((?:"[^"]*")|(?:(\S)+)))?)?)?'
        match = re.search(rex, line)
        classname = match.group(1)
        uid = match.group(2)
        attribute = match.group(3)
        value = match.group(4)
        if not match:
            print("** class name missing **")
        elif classname not in storage.classes():
            print("** class doesn't exist **")
        elif uid is None:
            print("** instance id missing **")
        else:
            key = "{}.{}".format(classname, uid)
            if key not in storage.all():
                print("** no instance found **")
            elif not attribute:
                print("** attribute name missing **")
            elif not value:
                print("** value missing **")
            else:
                cast = None
                if not re.search('^".*"$', value):
                    if '.' in value:
                        cast = float
                    else:
                        cast = int
                else:
                    value = value.replace('"', '')
                attributes = storage.attributes()[classname]
                if attribute in attributes:
                    value = attributes[attribute](value)
                elif cast:
                    try:
                        value = cast(value)
                    except ValueError:
                        pass  # fine, stay a string then
                setattr(storage.all()[key], attribute, value)
                storage.all()[key].save()


if __name__ == '__main__':
    HBNBCommand().cmdloop()

(hbnb) quit


# Tests

In [ ]:
my_model = BaseModel()
my_model.name = "My First Model"
my_model.my_number = 89
print(my_model)
my_model.save()
print(my_model)
my_model_json = my_model.to_dict()
print(my_model_json)
print("JSON of my_model:")
for key in my_model_json.keys():
    print("\t{}: ({}) - {}".format(key, type(my_model_json[key]), my_model_json[key]))

[BaseModel] (5b5791cc-1e19-4159-9b60-e5f259d2e581) {'id': '5b5791cc-1e19-4159-9b60-e5f259d2e581', 'created_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 697976), 'updated_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 697980), 'name': 'My First Model', 'my_number': 89}
[BaseModel] (5b5791cc-1e19-4159-9b60-e5f259d2e581) {'id': '5b5791cc-1e19-4159-9b60-e5f259d2e581', 'created_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 697976), 'updated_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 697980), 'name': 'My First Model', 'my_number': 89, 'update_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 698341)}
{'id': '5b5791cc-1e19-4159-9b60-e5f259d2e581', 'created_at': '2023-03-11T22:56:11.697976', 'updated_at': '2023-03-11T22:56:11.697980', 'name': 'My First Model', 'my_number': 89, 'update_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 698341), '__class__': 'BaseModel'}
JSON of my_model:
	id: (<class 'str'>) - 5b5791cc-1e19-4159-9b60-e5f259d2e581
	created_at: (<class 'str'>) - 2023-03-11T

In [ ]:
my_model = BaseModel()
my_model.name = "My_First_Model"
my_model.my_number = 89
print(my_model.id)
print(my_model)
print(type(my_model.created_at))
print("--")
my_model_json = my_model.to_dict()
print(my_model_json)
print("JSON of my_model:")
for key in my_model_json.keys():
    print("\t{}: ({}) - {}".format(key, type(my_model_json[key]), my_model_json[key]))

print("--")
my_new_model = BaseModel(**my_model_json)
print(my_new_model.id)
print(my_new_model)
print(type(my_new_model.created_at))

print("--")
print(my_model is my_new_model)

ad9f1e8d-39ae-4f80-8192-0393d5785fb3
[BaseModel] (ad9f1e8d-39ae-4f80-8192-0393d5785fb3) {'id': 'ad9f1e8d-39ae-4f80-8192-0393d5785fb3', 'created_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 727082), 'updated_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 727086), 'name': 'My_First_Model', 'my_number': 89}
<class 'datetime.datetime'>
--
{'id': 'ad9f1e8d-39ae-4f80-8192-0393d5785fb3', 'created_at': '2023-03-11T22:56:11.727082', 'updated_at': '2023-03-11T22:56:11.727086', 'name': 'My_First_Model', 'my_number': 89, '__class__': 'BaseModel'}
JSON of my_model:
	id: (<class 'str'>) - ad9f1e8d-39ae-4f80-8192-0393d5785fb3
	created_at: (<class 'str'>) - 2023-03-11T22:56:11.727082
	updated_at: (<class 'str'>) - 2023-03-11T22:56:11.727086
	name: (<class 'str'>) - My_First_Model
	my_number: (<class 'int'>) - 89
	__class__: (<class 'str'>) - BaseModel
--
ad9f1e8d-39ae-4f80-8192-0393d5785fb3
[BaseModel] (ad9f1e8d-39ae-4f80-8192-0393d5785fb3) {'id': 'ad9f1e8d-39ae-4f80-8192-0393d5785fb3', 'created_a

In [ ]:
all_objs = storage.all()
print("-- Reloaded objects --")
for obj_id in all_objs.keys():
    obj = all_objs[obj_id]
    print(obj)

print("-- Create a new object --")
my_model = BaseModel()
my_model.name = "My_First_Model"
my_model.my_number = 89
my_model.save()
print(my_model)

-- Reloaded objects --
[BaseModel] (5b5791cc-1e19-4159-9b60-e5f259d2e581) {'id': '5b5791cc-1e19-4159-9b60-e5f259d2e581', 'created_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 697976), 'updated_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 697980), 'name': 'My First Model', 'my_number': 89, 'update_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 698341)}
[BaseModel] (ad9f1e8d-39ae-4f80-8192-0393d5785fb3) {'id': 'ad9f1e8d-39ae-4f80-8192-0393d5785fb3', 'created_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 727082), 'updated_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 727086), 'name': 'My_First_Model', 'my_number': 89}
-- Create a new object --
[BaseModel] (e76cdf0c-15f1-412a-be3c-1743e9675318) {'id': 'e76cdf0c-15f1-412a-be3c-1743e9675318', 'created_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 747294), 'updated_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 747297), 'name': 'My_First_Model', 'my_number': 89, 'update_at': datetime.datetime(2023, 3, 11, 22, 56, 11, 747441)}
